# Google Drive Access

In [1]:
# Configure access to Google Colab folder to save results later
# Reference: https://hackmd.io/@BrownDeepLearningS24/BkIT3sW6s

import os
import sys

results_path = ''

isColab = "google.colab" in sys.modules
if isColab:
  from google.colab import drive
  results_path = ("/content/drive" + "/My Drive/Colab Notebooks/")
  sys.path.append(results_path)

## Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME'], axis=1, inplace=True)

In [4]:
# Determine the number of unique values in each column.
unique_values_count_per_column = application_df.nunique(axis=0)
print(unique_values_count_per_column)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [5]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_value_counts = application_df.groupby(['APPLICATION_TYPE']).size()
print(application_type_value_counts.sort_values(ascending=False))

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
dtype: int64


In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

# Chosen cutoff value is 500:
application_types_to_replace = application_type_value_counts[application_type_value_counts < 500].index.to_list()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
revised_application_type_value_counts = application_df['APPLICATION_TYPE'].value_counts()
print(revised_application_type_value_counts.sort_values(ascending=False))

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64


In [7]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_type_value_counts = application_df.groupby(['CLASSIFICATION']).size()
print(classification_type_value_counts.sort_values(ascending=False))

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2190        1
C2380        1
C2500        1
C2561        1
C8210        1
Length: 71, dtype: int64


In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_value_counts = application_df.groupby(['CLASSIFICATION']).size()
classification_value_counts_gt_1 = classification_value_counts[classification_value_counts > 1]
print(classification_value_counts_gt_1.sort_values(ascending=False))

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1278       10
C1238       10
C1235        9
C1237        9
C7210        7
C1720        6
C4100        6
C2400        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C1256        2
C1246        2
C3200        2
C1267        2
C1234        2
dtype: int64


In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

# Chosen cutoff value is 1000:
classifications_to_replace = classification_value_counts[classification_value_counts < 1000].index.to_list()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
revised_classification_value_counts = application_df['CLASSIFICATION'].value_counts()
print(revised_classification_value_counts.sort_values(ascending=False))

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64


In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_columns = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']

encoded_application_df = pd.get_dummies(data=application_df, columns=categorical_columns, dtype=int)


In [11]:
# Split our preprocessed data into our features and target arrays
y = encoded_application_df['IS_SUCCESSFUL']              # target
X = encoded_application_df.drop(columns='IS_SUCCESSFUL') # features

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=1,
                                                    stratify=y)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
# Inspect inputs to check the number of feature inputs (columns)
X.head()

,STATUS,ASK_AMT,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


## Compile, Train and Evaluate the Model

### Optimisation 1 - Use a different activation function

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="tanh", input_dim=43))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="tanh"))

# Output layer
# Define 1 neuron in the output layer (to determine a binary output i.e. Successful/Unsuccessful)
# Expected number of parameters for this layer = number of neurons in previous layer, plus 1 bias input, so 30+1=31.
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 80)                  │           3,520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 30)                  │           2,430 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              31 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,981 (23.36 KB)

 Trainable params: 5,981 (23.36 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Define folder path to save intermediate/final model weights
opt1_checkpoint_filepath = results_path + 'opt1_checkpoint.weights.h5'

# Define a callback function we can use to save the model weights every 5 epochs
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=opt1_checkpoint_filepath, save_weights_only=True, save_freq=5)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, callbacks=[checkpoint], epochs=100, verbose=2)

Epoch 1/100
804/804 - 11s - 13ms/step - accuracy: 0.7177 - loss: 0.5730
Epoch 2/100
804/804 - 4s - 5ms/step - accuracy: 0.7284 - loss: 0.5575
Epoch 3/100
804/804 - 6s - 7ms/step - accuracy: 0.7297 - loss: 0.5530
Epoch 4/100
804/804 - 4s - 5ms/step - accuracy: 0.7309 - loss: 0.5505
Epoch 5/100
804/804 - 5s - 6ms/step - accuracy: 0.7303 - loss: 0.5489
Epoch 6/100
804/804 - 5s - 6ms/step - accuracy: 0.7316 - loss: 0.5484
Epoch 7/100
804/804 - 4s - 5ms/step - accuracy: 0.7325 - loss: 0.5470
Epoch 8/100
804/804 - 5s - 6ms/step - accuracy: 0.7311 - loss: 0.5460
Epoch 9/100
804/804 - 6s - 7ms/step - accuracy: 0.7326 - loss: 0.5458
Epoch 10/100
804/804 - 4s - 5ms/step - accuracy: 0.7337 - loss: 0.5450
Epoch 11/100
804/804 - 6s - 7ms/step - accuracy: 0.7341 - loss: 0.5443
Epoch 12/100
804/804 - 4s - 5ms/step - accuracy: 0.7347 - loss: 0.5441
Epoch 13/100
804/804 - 4s - 4ms/step - accuracy: 0.7345 - loss: 0.5428
Epoch 14/100
804/804 - 4s - 4ms/step - accuracy: 0.7346 - loss: 0.5429
Epoch 15/100


In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 2ms/step - accuracy: 0.7290 - loss: 0.5608
Loss: 0.5607641935348511, Accuracy: 0.7289795875549316


### Optimisation 2 - Add additional hidden layer

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=43))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Output layer
# Define 1 neuron in the output layer (to determine a binary output i.e. Successful/Unsuccessful)
# Expected number of parameters for this layer = number of neurons in previous layer, plus 1 bias input, so 30+1=31.
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 80)                  │           3,520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 30)                  │           2,430 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 10)                  │             310 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1)                   │              11 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,271 (24.50 KB)

 Trainable params: 6,271 (24.50 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
# Define folder path to save intermediate/final model weights
opt2_checkpoint_filepath = results_path + 'opt2_checkpoint.weights.h5'

# Define a callback function we can use to save the model weights every 5 epochs
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=opt2_checkpoint_filepath, save_weights_only=True, save_freq=5)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, callbacks=[checkpoint], epochs=100, verbose=2)

Epoch 1/100
804/804 - 6s - 7ms/step - accuracy: 0.7201 - loss: 0.5698
Epoch 2/100
804/804 - 6s - 7ms/step - accuracy: 0.7281 - loss: 0.5535
Epoch 3/100
804/804 - 5s - 6ms/step - accuracy: 0.7314 - loss: 0.5505
Epoch 4/100
804/804 - 5s - 6ms/step - accuracy: 0.7308 - loss: 0.5495
Epoch 5/100
804/804 - 5s - 7ms/step - accuracy: 0.7322 - loss: 0.5476
Epoch 6/100
804/804 - 4s - 5ms/step - accuracy: 0.7329 - loss: 0.5467
Epoch 7/100
804/804 - 4s - 5ms/step - accuracy: 0.7339 - loss: 0.5461
Epoch 8/100
804/804 - 6s - 8ms/step - accuracy: 0.7326 - loss: 0.5453
Epoch 9/100
804/804 - 4s - 5ms/step - accuracy: 0.7339 - loss: 0.5446
Epoch 10/100
804/804 - 4s - 6ms/step - accuracy: 0.7348 - loss: 0.5439
Epoch 11/100
804/804 - 5s - 7ms/step - accuracy: 0.7355 - loss: 0.5440
Epoch 12/100
804/804 - 4s - 5ms/step - accuracy: 0.7341 - loss: 0.5437
Epoch 13/100
804/804 - 6s - 8ms/step - accuracy: 0.7338 - loss: 0.5428
Epoch 14/100
804/804 - 4s - 5ms/step - accuracy: 0.7346 - loss: 0.5429
Epoch 15/100
80

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 2ms/step - accuracy: 0.7297 - loss: 0.5645
Loss: 0.5645050406455994, Accuracy: 0.72967928647995


### Optimisation 3 - Add additional neurons to hidden layers

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=43))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=40, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=20, activation="relu"))

# Output layer
# Define 1 neuron in the output layer (to determine a binary output i.e. Successful/Unsuccessful)
# Expected number of parameters for this layer = number of neurons in previous layer, plus 1 bias input, so 30+1=31.
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_7 (Dense)                      │ (None, 80)                  │           3,520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 40)                  │           3,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 20)                  │             820 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 1)                   │              21 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,601 (29.69 KB)

 Trainable params: 7,601 (29.69 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [24]:
# Define folder path to save intermediate/final model weights
opt3_checkpoint_filepath = results_path + 'opt3_checkpoint.weights.h5'

# Define a callback function we can use to save the model weights every 5 epochs
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=opt3_checkpoint_filepath, save_weights_only=True, save_freq=5)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, callbacks=[checkpoint], epochs=100, verbose=2)

Epoch 1/100
804/804 - 7s - 9ms/step - accuracy: 0.7188 - loss: 0.5703
Epoch 2/100
804/804 - 11s - 13ms/step - accuracy: 0.7302 - loss: 0.5541
Epoch 3/100
804/804 - 4s - 5ms/step - accuracy: 0.7309 - loss: 0.5510
Epoch 4/100
804/804 - 4s - 6ms/step - accuracy: 0.7311 - loss: 0.5497
Epoch 5/100
804/804 - 6s - 8ms/step - accuracy: 0.7325 - loss: 0.5481
Epoch 6/100
804/804 - 4s - 5ms/step - accuracy: 0.7313 - loss: 0.5474
Epoch 7/100
804/804 - 5s - 6ms/step - accuracy: 0.7318 - loss: 0.5465
Epoch 8/100
804/804 - 5s - 7ms/step - accuracy: 0.7333 - loss: 0.5463
Epoch 9/100
804/804 - 4s - 6ms/step - accuracy: 0.7328 - loss: 0.5453
Epoch 10/100
804/804 - 6s - 8ms/step - accuracy: 0.7334 - loss: 0.5452
Epoch 11/100
804/804 - 4s - 6ms/step - accuracy: 0.7328 - loss: 0.5445
Epoch 12/100
804/804 - 5s - 6ms/step - accuracy: 0.7347 - loss: 0.5443
Epoch 13/100
804/804 - 5s - 7ms/step - accuracy: 0.7355 - loss: 0.5435
Epoch 14/100
804/804 - 9s - 12ms/step - accuracy: 0.7352 - loss: 0.5428
Epoch 15/100

In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 2ms/step - accuracy: 0.7304 - loss: 0.5740
Loss: 0.5739877223968506, Accuracy: 0.7303789854049683


### Export final model results

In [26]:
# Export our model to HDF5 file
nn.save(results_path + 'AlphabetSoupCharity_Optimisation.h5')